## Econ 691-06
### Assignment 6

#### Armen Khachatrian
##### Collaboration with Joseph Oon

## Problem 1: Multi-Armed Bandits

AV  Club,  an  online  publisher  known  for  its  review  of  television  shows  and  movies,  is  debatingbetween two possible headlines for a short article.  Those two headlines are:

* Why the last episode of The Simpsons left us with a cliffhanger
* What the cliffhanger means for the next episode of The Simpsons

In both cases, the articles are the same (reviewing the last episode and forecasting the next episode);but the headline can either advertise its commentary on the last episode or on the next one.  Forthe sake of notation, suppose the test is the first headline (the last episode) and the control is thesecond headline (the next episode).  AV Club wants to deploy a bandit to maximize time spentreading the article.The  response  functions  for  article  readers  is  below,  in  both  Python  and  R.  These  represent  theamount  of  time,  in  seconds,  that  readers  spend  reading  the  article.   Notice  that  the  responsefunction differ from the one discussed in class in one crucial dimension:  the response to treatmentchanges over time.  Early readers have a relatively positive reaction to the first headline comparedto  the  second  one,  while  later  readers  have  a  relatively  negative  reaction.   This  should  not  besurprising, given that the first headline focuses on the last episode and so does best right after theepisode has been aired; and the second headline focuses on the next episode and so does best rightbefore that episode is to be aired.  In this simulation, we will always assume there are five hundredreaders who come to the article, soiranges from 1 to 500

> 1. (5 points) In class, we covered three approaches:  the standard experiment (in which eachuser  is  independently  assigned  to  treatment  with  50%  probability),  the  constant-$\epsilon$ bandit,and the declining-$\epsilon$ bandit.  Include them in your code, and update them in two ways.  First,update both their defaultnand their calls to the individualresponse function.  Second, havethe functionsonlyreturn the average time-spent, in seconds, on the article.  This is becausethe coefficient and standard error are no longer useful attributes of the data, because of thevariation in the coefficient across time and across the order of units.

In [473]:
#libraries
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.formula.api as sm
from scipy.stats import ttest_ind
#function
def individual_response(treatment, i):
    return(max(0, 12 + (5 - i/70) * treatment + np.random.normal(0, 5, 1)[0]))

In [474]:
treatment_effect = (np.mean([individual_response(1, i) for i in range(500)]) - 
                    np.mean([individual_response(0, i) for i in range(500)]))
print("The treatment effect is: " + str(round(treatment_effect, 4)))

The treatment effect is: 1.8923


In [475]:
# Define function experiment
def experiment(method = 'standard', n = 500, epsilon = 0.01, alpha = 0.001):
    if method == 'standard': #standard experimentation
        indicator = np.random.choice([0, 1], n, replace = True)
        response = []
        for i in range(n):
            response.append(individual_response(indicator[i], i))

        # response-treatment data
        data = pd.DataFrame({'response': response, 'treatment': indicator})
        #return average time-spent
        return round(sum(data['response'])/n, 4)
    
    if method == 'constant epsilon': #constant epsilon bandit
        response = np.array([])
        indicator = np.array([])
        for i in range(n):
            if np.random.rand(1)[0] < epsilon: #one random number/'explore' phase
                if np.random.rand(1)[0] < 0.5:
                    treatment = 1
                else:
                    treatment = 0
    
                # Else implement 'exploit' phase, where we take the best arm (greedy)
            else:
                y1 = sum(response[indicator == 1])/(sum(indicator) + 0.001)
                y2 = sum(response[indicator == 0])/(i - sum(indicator) + 0.001)
                if y1 > y2:
                    treatment = 1
                else:
                    treatment = 0
    
                # Get and save the individual's response
            response = np.append(response, individual_response(treatment, i))
            indicator = np.append(indicator, treatment)

        
        data = pd.DataFrame({'response': response, 'treatment': indicator})
        #return average time-spent
        return round(sum(data['response'])/n, 4)
        
    if method == 'decreasing epsilon': #decreasing epsilon bandit
        response = np.array([])
        indicator = np.array([])

      # Iterate through each person
        for i in range(n):
            if np.random.rand(1)[0] < np.exp(-alpha * i): #threshold
                if np.random.rand(1)[0] < 0.5:
                    treatment = 1
                else: 
                    treatment = 0
            else:#exploit
                y1 = sum(response[indicator == 1])/(sum(indicator) + 0.001)
                y2 = sum(response[indicator == 0])/(i - sum(indicator) + 0.001)
                if y1 > y2:
                    treatment = 1
                else:
                    treatment = 0

            response = np.append(response, individual_response(treatment, i))
            indicator = np.append(indicator, treatment)

        data = pd.DataFrame({'response': response, 'treatment': indicator})
        #return average time-spent
        return round(sum(data['response'])/n, 4)
    
    if method == 'thomson': #thomson sampling
        response = np.array([])
        indicator = np.array([])
        for i in range(30): #“burn-in”  period  
            if np.random.rand(1)[0] < 0.5:
                treatment = 1
            else:
                treatment = 0
            response = np.append(response, individual_response(treatment, i))
            indicator = np.append(indicator, treatment)
        for i in range(30, n):
            a = response[indicator == 1] #the set of treated
            b = response[indicator == 0] #the set of control
            t, pval = ttest_ind(a, b, equal_var=False) #calculating t value and p value
            p = norm.cdf(t) #converting t value to probability
            if np.random.rand(1)[0] < p: #random value
                treatment = 1
            else:
                treatment = 0
            response = np.append(response, individual_response(treatment, i))
            indicator = np.append(indicator, treatment)
        data = pd.DataFrame({'response': response, 'treatment': indicator}) 
        #return average time-spent
        return round(sum(data['response'])/n, 4)
# Set seed and run experiment
np.random.seed(10)
print('Standard experiment time-spent: ', experiment())
print('Constant epsilon bandit time-spent (default epsilon=0.01): ', experiment('constant epsilon'))
print('Decreasing epsilon bandit time-spent (default alpha=0.001): ', experiment('decreasing epsilon'))

Standard experiment time-spent:  12.7345
Constant epsilon bandit time-spent (default epsilon=0.01):  11.7717
Decreasing epsilon bandit time-spent (default alpha=0.001):  12.7405


----------

> 2. (25 points) AV Club also wants to implement a bandit that utilizes Thompson sampling, and asks for your assistance. Implement one, using the hints and guidance below; and demonstratethat it works by running the function once.

In [476]:
np.random.seed(10)
print('Thomson sampling time-spent: ', experiment('thomson'))

Thomson sampling time-spent:  13.4338


> 3. (10 points) Evaluate the constant-$\epsilon$ algorithm under four parameterizations: $\epsilon$= 0.5, $\epsilon$= 0.3, $\epsilon$= 0.1, and $\epsilon$= 0.01.  To mitigate noise, run each parameterization on one hundred differentstreams of data, and take the average of the hundred runs per parameterization. Explain the results:  which $\epsilon$ performs best, which $\epsilon$ performs worst, and why?

In [477]:
lst = []
mean_values = []
epsilons = [0.5, 0.3, 0.1, 0.01]
for eps in epsilons:
    for _ in range(100):
        lst.append(experiment(method = 'constant epsilon', n = 500, epsilon = eps))
    mean_values.append(np.mean(lst)) 
    lst.clear()
np.random.seed(10)
mean_values

[13.062985000000001, 13.171197999999999, 13.14611, 12.503052000000002]

**Answer:** $\epsilon$ = 0.3 performs the best as it gives us the highest average time on site per person. 
$\epsilon$ = 0.01 performs the worse. 
This is the first if loop in our experiment function has the condition if np.random.rand(1)[0] < $\epsilon$ and having 0.01 as our $\epsilon$ is very aggressive, and we will go, in general, to the exploitation phase. We can explain it because the threshold ($\epsilon$) is low enough (0.01). The fact that we ignore the exploration phase will cause that y1 is larger than y2 (choosing the best arm), and it leads that most of the observations will go to the treatment group. When $\epsilon$ equals 0.3, we obtain the highest average time with a 30% probability of being explored.

> 4. (10 points) Evaluate  the  declining-$\epsilon$ algorithm  under  three  parameterizations: $\alpha$ =  0.01, $\alpha$ = 0.005, and $\alpha$ =  0.001.   To  mitigate  noise,  run  each  parameterization  on  one  hundred different  streams  of  data,  and  take  the  average  of  the  hundred  runs  per parameterization. Explain the results:  whichαperforms best, which $\alpha$ performs worst, and why?

In [478]:
lst2 = []
mean_values2 = []
alphas = [0.01, 0.005, 0.001]
for alpha in alphas:
    for _ in range(100):
        lst2.append(experiment(method = 'decreasing epsilon', n = 500, alpha = alpha))
    mean_values2.append(np.mean(lst2)) 
    lst2.clear()
np.random.seed(10)
mean_values2 

[13.082727000000004, 12.924902000000001, 12.754246000000002]

**Answer:** The best $\alpha$ is 0.01 and the worst $\alpha$ is 0.001. 

Going back to our experiment function the first if condition is np.random.rand(1)[0] < np.exp(-alpha * i)  the smaller the $\alpha$ actually makes np.exp(-alpha*i) really close to 1 and that will mean that all our observations will only go through the exploration phase and the experiment will be no different from a standard experiment as no observations will go through the multi arm bandit section.

> 5. (5 points) Evaluate the four approaches:  the experiment, the best constant-$\epsilon$ algorithm, the best declining-$\epsilon$ algorithm, and Thompson sampling.  To mitigate noise, run each algorithm on one  hundred different streams of data, and take the average of the hundred runs per algorithm.

In [479]:
methods = ['standard', 'constant epsilon', 'decreasing epsilon', 'thomson']
lst3 = []
mean_values3 = []
for method in methods:
    for _ in range(100):
        lst3.append(experiment(method = method, epsilon = 0.3, alpha = 0.01))
    mean_values3.append(np.mean(lst3)) 
    lst3.clear()
np.random.seed(10)
np.round(mean_values3, 4)

array([12.7324, 13.1423, 13.1078, 13.3283])

**Answer:** \
Standard Experiment: 12.7324 sec\
Constant-$\epsilon$ bandit: 13.1423 sec\
Decreasing-$\epsilon$ bandit: 13.1078 sec\
Thomson sampling: 13.3283 sec

> 6. Using these results, answer the following questions comparing Thompson sampling to each of the algorithms in turn.\
    (a) (5 points) Compare Thompson sampling to the experiment.  Which performs betterand why?\
    (b) (5 points) Compare  Thompson  sampling  to  the  best  constant-$\epsilon$ algorithm. Which performs better and why?\
    (c) (5 points) Compare  Thompson  sampling  to  the  best  declining-$\epsilon$ algorithm.   Which performs better and why?

a) The Thompson  sampling performs better standard experiment showing a 0.5 more seconds spent on site per user. This is because in a standard experiment we are unable to explore and exploit at the same time. Using the Thompson sampling method we are able to.  Due to the nature of this experiment the time frame is short, we do not have time to explore fist and exploit later. In this experiment which was defined by the `individual_response` function where early reader have positive reaction to the first heading and later readers have negative. With shifting parameters like this Thompson sampling will perform better.

b) Between constant epsilon and thompson sampling, thompson sampling performs better. thompson sampling performs better because with the weighted arm approach, our observation converges to the  optimal arm over time and this weights can change depending on our observations.

c) Between decreasing epsilon and thompson sampling, thompson sampling performs better. Since we have an environment that evolves over time, constant epsilon performs better than decreasing epsilon and that can be observed from our result. However, the reason why thompson sampling performs better is because with a changing environment like this experiment, a thompson sampling can adjust accordingly, but a decreasing epsilon method cannot go back to a high exploration mode once the underlying enivornment changes.